<a href="https://colab.research.google.com/gist/HashemMZ/08584a3c091bf3953c9ffdb1befdd706/ch3-regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# chapter 3 - regression
# boston_reg.py
# Python 3.5.2, TensorFlow 2.1.5, Keras 1.7.0
import numpy as np
import tensorflow as tf
from tensorflow import keras as K
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

In [ ]:
class MyLogger(K.callbacks.Callback):
  def __init__(self, n, data_x, data_y, pct_close):
    self.n = n
    self.data_x = data_x
    self.data_y = data_y
    self.pct_close = pct_close

  def on_epoch_end(self, epoch, logs={}):
    if epoch % self.n == 0:
      curr_loss = logs.get('loss')
      total_acc = my_accuracy(self.model, self.data_x, self.data_y, self.pct_close)
      print("epoch = %4d curr batch loss (mse) = %0.6f overall acc = %0.2f%%" % \
      (epoch, curr_loss, total_acc * 100))

def my_accuracy(model, data_x, data_y, pct_close):
  num_correct = 0; num_wrong = 0
  n = len(data_x)
  for i in range(n):
    predicted = model.predict(np.array([data_x[i]], dtype=np.float32)) #[[x]]
    actual = data_y[i]
    if np.abs(predicted[0][0] - actual) < np.abs(pct_close * actual):
      num_correct += 1
    else:
      num_wrong += 1
  return (num_correct * 1.0) / (num_correct + num_wrong)

In [ ]:
# 0. get started
print("\nBoston Houses dataset regression example ")
np.random.seed(2)
tf.random.set_seed(3)
kv = K.__version__
print("Using Keras: ", kv, "\n")



Boston Houses dataset regression example 
Using Keras:  2.8.0 



In [ ]:
# 1. load data
!git clone 'https://github.com/HashemMZ/keras-succinctly.git'
print("Loading Boston data into memory ")
data_file = "/content/keras-succinctly/Boston/boston_mm_tab.txt"
all_data = np.loadtxt(data_file, delimiter="\t", skiprows=0, dtype=np.float32)
N = len(all_data)
indices = np.arange(N)
np.random.shuffle(indices)
n_train = int(0.80 * N)
print("Splitting data into training and test sets \n")
data_x = all_data[indices,:-1]
data_y = all_data[indices,-1]
train_x = data_x[0:n_train,:]
train_y = data_y[0:n_train]
test_x = data_x[n_train:N,:]
test_y = data_y[n_train:N]

Cloning into 'datasets4keras'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 6 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (6/6), done.
Loading Boston data into memory 
Splitting data into training and test sets 



In [ ]:
# 2. define model
init = K.initializers.RandomUniform(seed=1)
simple_sgd = K.optimizers.SGD(learning_rate=0.010)#پارامتر نرخ یادگیری اسم جدیدتری دارد
model = K.models.Sequential()
model.add(K.layers.Dense(units=10, input_dim=13, kernel_initializer=init, activation='tanh')) # hidden layer
model.add(K.layers.Dense(units=10, kernel_initializer=init, activation='tanh')) # hidden layer
model.add(K.layers.Dense(units=1, kernel_initializer=init, activation=None))
model.compile(loss='mean_squared_error', optimizer=simple_sgd, metrics=['mse'])


In [ ]:
# 3. train model
batch_size= 8
max_epochs = 500
my_logger = MyLogger(int(max_epochs/5), train_x, train_y, 0.15)
print("Starting training ")
h = model.fit(train_x, train_y, batch_size=batch_size, epochs=max_epochs, verbose=0, callbacks=[my_logger])
print("Training finished \n")

Starting training 
epoch =    0 curr batch loss (mse) = 0.928677 overall acc = 37.87%
epoch =  100 curr batch loss (mse) = 0.192066 overall acc = 68.07%
epoch =  200 curr batch loss (mse) = 0.164511 overall acc = 70.79%
epoch =  300 curr batch loss (mse) = 0.142651 overall acc = 72.52%
epoch =  400 curr batch loss (mse) = 0.131459 overall acc = 71.78%
Training finished 



In [ ]:
# 4. evaluate model
acc = my_accuracy(model, train_x, train_y, 0.15)
print("Overall accuracy (wthin 15%%) on training data = %0.4f" % acc)
acc = my_accuracy(model, test_x, test_y, 0.15)
print("Overall accuracy on test data = %0.4f \n" % acc)
eval = model.evaluate(train_x, train_y, verbose=0)
print("Overall loss (mse) on training data = %0.6f" % eval[0])
eval = model.evaluate(test_x, test_y, verbose=0)
print("Overall loss (mse) on test data = %0.6f" % eval[0])



Overall accuracy (wthin 15%) on training data = 0.6634
Overall accuracy on test data = 0.6961 

Overall loss (mse) on training data = 0.132403
Overall loss (mse) on test data = 0.122609


In [ ]:
# 5. save model
print("\nSaving Boston model to disk \n")
os.chdir('/content/sample_data')
mp = "boston_model.h5"
model.save(mp)




Saving Boston model to disk 



In [ ]:
# 6. use model
np.set_printoptions(precision=1)
unknown = np.full(shape=(1,13), fill_value=0.6, dtype=np.float32) 
unknown[0][3] = -1.0 # binary feature
predicted = model.predict(unknown)
print("Using model to predict median house price for features: ")
print(unknown)
print("\nPredicted price is: ")
print("$%0.2f" % (predicted * 10000))

Using model to predict median house price for features: 
[[ 0.6  0.6  0.6 -1.   0.6  0.6  0.6  0.6  0.6  0.6  0.6  0.6  0.6]]

Predicted price is: 
$9411.44
